In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
votes = pd.DataFrame.from_csv('AnsVotes_TSeries.csv', index_col=None)
votes.head(15)

,QuestionId,AnsId,Age_vot,AnsWordCount,AcceptedAnsId,QScore,QVotes,Score,Votes,CumVotes,CumScore,ReScore,QReScore,Comments,AnsRank,ReScore_rank,Ans_count,Pbias,DRank,EPbias
0,1,56,0,201,56,0,0,2,2,2,2,0.666667,0.000,0,1,1,1,1,0,1
1,2,10,0,145,10,0,0,3,3,3,3,0.750000,0.000,0,1,1,1,1,0,1
2,2,10,0,145,10,0,0,3,3,3,3,0.750000,0.000,0,1,1,1,1,0,1
3,3,9,0,25,109,0,0,4,4,4,4,0.800000,0.000,0,1,1,1,1,0,1
4,7,16,0,45,2977,0,0,3,3,3,3,0.750000,0.000,0,1,1,1,1,0,1
5,14,20,0,28,20,0,0,7,7,7,7,0.875000,0.000,0,1,1,1,1,0,1
6,18,33,0,133,33,0,0,3,3,3,3,0.750000,0.000,0,1,1,1,1,0,1
7,18,33,0,133,33,0,0,3,3,3,3,0.750000,0.000,0,1,1,1,1,0,1
8,18,33,0,133,33,0,0,3,3,3,3,0.750000,0.000,0,1,1,1,1,0,1
9,21,80,0,43,80,0,0,1,1,1,1,0.500000,0.000,0,1,1,1,1,0,1


In [3]:
votes[['Age_vot','AnsWordCount','Comments','Ans_count']].describe()

,Age_vot,AnsWordCount,Comments,Ans_count
count,23328.000000,23328.000000,23328.000000,23328.000000
mean,212.595508,143.485039,2.033522,2.721622
std,380.532603,150.314528,2.851866,2.061537
min,0.000000,3.000000,0.000000,1.000000
25%,1.000000,51.000000,0.000000,1.000000
50%,7.000000,97.000000,1.000000,2.000000
75%,231.000000,178.000000,3.000000,3.000000
max,1504.000000,2142.000000,18.000000,12.000000


In [4]:
votes['Ans_count'].value_counts().head(15)

1     7132
2     6866
3     4033
4     2172
5     1169
6      535
10     484
7      283
8      273
9      240
11      73
12      68
Name: Ans_count, dtype: int64

In [5]:
#votes = votes[ votes.age_vot>=3]
votes_r = votes[votes.Age_vot<=7]
votes_r[['Score','Votes','CumScore','CumVotes','QScore','QVotes','ReScore','QReScore','Ans_count','DRank']].describe()

,Score,Votes,CumScore,CumVotes,QScore,QVotes,ReScore,QReScore,Ans_count,DRank
count,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000,11949.000000
mean,1.674785,1.787765,3.338020,3.505733,4.262030,4.482802,0.667576,0.523072,1.892125,0.069462
std,1.404004,1.326312,2.641695,2.575425,6.957248,7.212644,0.233654,0.380433,1.204971,0.589805
min,-3.000000,1.000000,-6.000000,1.000000,-7.000000,0.000000,-0.857143,-0.875000,1.000000,-4.000000
25%,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.500000,0.000000,1.000000,0.000000
50%,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.750000,0.666667,2.000000,0.000000
75%,2.000000,2.000000,5.000000,5.000000,6.000000,6.000000,0.833333,0.833333,2.000000,0.000000
max,12.000000,12.000000,18.000000,18.000000,58.000000,62.000000,0.947368,0.979592,9.000000,8.000000


# Votes/day on a given answer . (Context information: question, ans aggregates)

In [99]:
#Comments Ans_count AnsWordCount DRank Age_vot 
mod_ols = smf.ols(formula='np.log(Votes+1) ~ ReScore + EPbias + Score -1', data=votes_r)#&(votes.DRank==2)])
res = mod_ols.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:      np.log(Votes + 1)   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                 6.037e+04
Date:                Sat, 02 Apr 2016   Prob (F-statistic):               0.00
Time:                        03:10:22   Log-Likelihood:                -506.22
No. Observations:               11949   AIC:                             1018.
Df Residuals:                   11946   BIC:                             1041.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
ReScore        0.3272      0.008     40.785      0.0

In [98]:
mod_poi = smf.glm('Votes ~ ReScore + EPbias + Score  - 1 ', data=votes,
                  family=sm.families.Poisson(link=sm.families.links.log)).fit(method='bfgs') #
#print mod_poi.mle_retvals['converged']
print mod_poi.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Votes   No. Observations:                23328
Model:                            GLM   Df Residuals:                    23325
Model Family:                 Poisson   Df Model:                            2
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -26890.
Date:                Sat, 02 Apr 2016   Deviance:                       2634.0
Time:                        03:10:15   Pearson chi2:                 2.62e+03
No. Iterations:                     8                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
ReScore       -0.3761      0.016    -24.106      0.000        -0.407    -0.346
EPbias         0.1970      0.017     11.755      0.0

## How does question quality relate to answer quality

In [108]:
#Comments Ans_count AnsWordCount DRank Age_vot
four_ans = smf.ols(formula='QReScore ~ ReScore + C(AnsRank,Helmert) - 1 ', data=votes[votes['Ans_count']==4])
re_four = four_ans.fit()
print re_four.summary()

                            OLS Regression Results                            
Dep. Variable:               QReScore   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     46.23
Date:                Sat, 02 Apr 2016   Prob (F-statistic):           2.53e-37
Time:                        03:22:38   Log-Likelihood:                 331.40
No. Observations:                2172   AIC:                            -652.8
Df Residuals:                    2167   BIC:                            -624.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------------
C(AnsRan

In [94]:
mod_poi = smf.glm('QReScore ~ ReScore + EPbias - 1 ', data=votes[votes['Ans_count']==4],
                  family=sm.families.Poisson(link=sm.families.links.log)).fit(method='bfgs') #
#print mod_poi.mle_retvals['converged']
print mod_poi.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:               QReScore   No. Observations:                 2172
Model:                            GLM   Df Residuals:                     2170
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -2016.8
Date:                Sat, 02 Apr 2016   Deviance:                       168.97
Time:                        03:07:59   Pearson chi2:                     143.
No. Iterations:                     7                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
ReScore        0.0081      0.067      0.121      0.904        -0.123     0.139
EPbias        -0.7037      0.172     -4.091      0.0

In [145]:
levels = [1,2,3,4]
contrast = KRanks().code_without_intercept(levels)
print contrast.matrix

[[ 3.  0.  0.]
 [-1.  2.  0.]
 [-1. -1.  1.]
 [-1. -1. -1.]]


In [144]:
class KRanks(object):
    """K-rank contrasts.
       Compare relative effect on output variable for a given level
       against all those below.
    """
    def _kranks_contrast(self, levels):
        n = len(levels)
        contr = np.zeros((n, n - 1))
        contr[:-1][np.diag_indices(n - 1)] = np.arange(1, n)[::-1]
        rows,cols = np.tril_indices(n-1)
        contr[(rows+1,cols)] = -1
        return contr

    def code_with_intercept(self, levels):
        contrast = np.column_stack((np.ones(len(levels)),
                                    self._kranks_contrast(levels)))
        column_suffixes = _name_levels("K-r.", ["intercept"] + list(levels[1:]))
        return ContrastMatrix(contrast, column_suffixes)

    def code_without_intercept(self, levels):
        contrast = self._kranks_contrast(levels)
        return ContrastMatrix(contrast, _name_levels("K-r.", levels[1:]))
                              